Installations of Lego Segmentation Code

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!git clone https://github.com/gauravdesale8/Mask_RCNN/blob/master/calculate_object_area.ipynp
import sys
sys.path.append("/content/calculate_object_area/mrcnn")
from mrcnn_colab_engine import *
!wget https://drive.google.com/file/d/1Q4RvTpiHgtYMR8km_u46ZAOPlL83AFhw/view?usp=sharing
!wget -O lego_image_sample.jpg https://drive.google.com/file/d/1o24hif1_u_CNYyd_cTSntjsBo5YAbQvz/view?usp=sharing
MODEL_DIR = "/content/mask_rcnn_object_0009-220120-123759.h5"
# Load Mask rcnn model
config = InferenceConfig()
lego_model = modellib.MaskRCNN(mode="inference", model_dir="", config=config)
# Load weights generated to detect 4 lego categories
lego_model.load_weights(MODEL_DIR, by_name=True)

# Mask RCNN model + Functions
from google.colab.patches import cv2_imshow
from mrcnn_colab_engine import random_colors, draw_mask, detect_contours_maskrcnn
class_names = ["BG", "green", "blue", "light blue", "pink"]
# Generate random colors
colors = random_colors(len(class_names))

Loading the image and detecting Legos area

In [ ]:
RATIO_PIXEL_TO_CM = 78 # 78 pixels are 1cm
RATIO_PIXEL_TO_SQUARE_CM = 78 * 78

In [ ]:
import cv2
from mrcnn_colab_engine import detect_contours_maskrcnn, draw_mask

img = cv2.imread("lego_image_sample.jpg")

# 1. Get objects mask with Mask RCNN
class_ids, boxes, masks = detect_contours_maskrcnn(lego_model, img)
for class_id, box, object_contours in zip(class_ids, boxes, masks):
    # Box
    y1, x1, y2, x2 = box
    # cv2.rectangle(img, (x1, y1), (x2, y2), colors[class_id], 15)
    cv2.polylines(img, [object_contours], True, colors[class_id], 10)
    img = draw_mask(img, [object_contours], colors[class_id])

    # 2. Calculate area
    area_px = cv2.contourArea(object_contours)
    area_cm = round(area_px / RATIO_PIXEL_TO_SQUARE_CM, 2)

    cv2.putText(img, "A: {}cm^2".format(area_cm), (x1, y1), cv2.FONT_HERSHEY_PLAIN, 3, colors[class_id], 5)


cv2_imshow(img)